# Momento de Retroalimentación: Módulo 1 Técnicas de procesamiento de datos para el análisis estadístico y para la construcción de modelos (Portafolio Análisis)


It is well known that salaries has changed in comparison with the amount of jobs. Noweadays the problem is that people can't focus in reaching an specefic possition or get a new kind of job related to the future of technology. That's the reason why we have created this study, to demostrate and compare the diferences between jobs and salaries.

In [ ]:
#install 
!pip install country_converter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import country_converter as coco

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from wordcloud import WordCloud

import nltk

In [ ]:
%matplotlib inline
sns.set_theme(style="dark")
mpl.rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns',None)
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

# Data Exploration

In [ ]:
df = pd.read_csv('/content/ds_salaries.csv')
df.drop(df[['salary','salary_currency','Unnamed: 0']],axis=1, inplace=True)

In [ ]:
df.shape
df.tail(5)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
602,2022,SE,FT,Data Engineer,154000,US,100,US,M
603,2022,SE,FT,Data Engineer,126000,US,100,US,M
604,2022,SE,FT,Data Analyst,129000,US,0,US,M
605,2022,SE,FT,Data Analyst,150000,US,100,US,M
606,2022,MI,FT,AI Scientist,200000,IN,100,US,L


# Variable Type

In [ ]:
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

# Cualitative 
company_location      object
company_size          object
experience_level      object
employment_type       object
job_title             object
employee_residence    object




In [ ]:
cual = df[['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location', 'company_size', 'work_year']]
cols = cual.columns
for i in range(1, cols.size):  
    print(cual.groupby(cols[i]).agg(frequency=(cols[i], "count")))

                 frequency
employment_type           
CT                       5
FL                       4
FT                     588
PT                      10
                                          frequency
job_title                                          
3D Computer Vision Researcher                     1
AI Scientist                                      7
Analytics Engineer                                4
Applied Data Scientist                            5
Applied Machine Learning Scientist                4
BI Data Analyst                                   6
Big Data Architect                                1
Big Data Engineer                                 8
Business Data Analyst                             5
Cloud Data Engineer                               2
Computer Vision Engineer                          6
Computer Vision Software Engineer                 3
Data Analyst                                     97
Data Analytics Engineer                           4
Data A

# Cuantitative
salary_in_usd          int64

# What is the salary that a data analyst can aspire to?
Accordin to the mean the answer to this question is 112297.869852

In [ ]:
cuan = df[['salary_in_usd']]

In [ ]:
print("Mean: ")
print(cuan.mean())
print(" ")
print("Median: ")
print(cuan.median())
print(" ")
print("Mode: ")
print(cuan.mode())
print(" ")


Mean: 
salary_in_usd    112297.869852
dtype: float64
 
Median: 
salary_in_usd    101570.0
dtype: float64
 
Mode: 
   salary_in_usd
0         100000
 


In [ ]:
print("Range: ")
print(cuan.max() - cuan.min())
print("")
print("Variance: ")
print(cuan.var())
print("")
print("Standart Desviation: ")
print(cuan.std())
print("")

Range: 
salary_in_usd    597141
dtype: int64

Variance: 
salary_in_usd    5.034933e+09
dtype: float64

Standart Desviation: 
salary_in_usd    70957.259411
dtype: float64



In [ ]:
fig1 = px.box(y=df['salary_in_usd'],template= 'plotly_dark', title = 'BoxPlot of Salary in USD (Cuantitative)')
fig1.update_layout(font = dict(size=17,family="Franklin Gothic"))
fig1.show()

# The level of experience influence the salary?
Given this data we can know the experience level of each person and their respective percentage amount of workers around the worl. 


In [ ]:
df['experience_level'] = df['experience_level'].replace('EN','Entry-level/Junior')
df['experience_level'] = df['experience_level'].replace('MI','Mid-level/Intermediate')
df['experience_level'] = df['experience_level'].replace('SE','Senior-level/Expert')
df['experience_level'] = df['experience_level'].replace('EX','Executive-level/Director')

ex_level = df['experience_level'].value_counts()
fig = px.treemap(ex_level, 
                 path=[ex_level.index],
                 values=ex_level.values, 
                 title = 'Experience Level',
                 color=ex_level.index,
                 color_discrete_sequence=px.colors.sequential.PuBuGn,
                 template='plotly_dark',
#                  textinfo = "label+value+percent parent+percent entry+percent root",
                 width=1000, height=500)

percents = np.round((100*ex_level.values / sum(ex_level.values)).tolist(),2)
print(percents)
en = len(df.query("experience_level == 'Entry-level/Junior'"))
print(en)


fig.data[0].customdata = [35.09, 46.13, 4.28 , 14.5]
fig.data[0].texttemplate = '%{label}<br>%{value}<br>%{customdata}%'

fig.update_layout(
    font=dict(size=17,family="Franklin Gothic"))

fig.show()

[46.13 35.09 14.5   4.28]
88


In [ ]:
current_salary = df.groupby(['salary_in_usd','experience_level']).size().reset_index()
current_average = current_salary.groupby('experience_level').mean().reset_index()
current_average = current_average.sort_values(by=['salary_in_usd'])

fig3 = px.bar(x=current_average['experience_level'],y=current_average['salary_in_usd'],text = current_average['salary_in_usd'], 
                   color = current_average['salary_in_usd'], color_discrete_sequence=px.colors.sequential.PuBu)

fig3.update_layout(
    xaxis_title="Experience Level",
    yaxis_title="Mean Salary")
fig3.update_layout(barmode = 'relative',xaxis_tickangle=-45, 
                  title='Average Salary by Experience Level', template='plotly_dark',font = dict(size=17,family="Franklin Gothic"))
fig3.show()

# Which country offers the best salary?
To answer this question it is necessary to know the average salary by country. Since all salaries can change in different countries, we use "Salary_in_USD" to calculate the average for any country.

In [ ]:
converted_country = coco.convert(names=df['company_location'], to="ISO3")
df['company_location'] = converted_country

salary_location = df.groupby(['salary_in_usd','company_location']).size().reset_index()
#print(salary_location)

average = salary_location.groupby('company_location').mean().reset_index()
#print(average)
fig = px.choropleth(locations=average['company_location'],
                    color=average['salary_in_usd'],
                    color_continuous_scale=px.colors.sequential.PuBu,
                    template='plotly_dark',
                    title = 'Mean of Salaries in USD by Company Location')
fig.update_layout(font = dict(size=17,family="Franklin Gothic"))
fig.show()

In [ ]:
average = average.sort_values(by=['salary_in_usd'])[-10:]
fig = px.bar(x=average['company_location'],y=average['salary_in_usd'],text = average['salary_in_usd'], 
                   color = average['salary_in_usd'], color_discrete_sequence=px.colors.sequential.PuBu)

fig.update_layout(
    xaxis_title="Countries",
    yaxis_title="Mean Salary")
fig.update_layout(barmode = 'relative',xaxis_tickangle=-45, 
                  title='Top 10 Highest Salary by Country', template='plotly_dark',font = dict(size=17,family="Franklin Gothic"))

Given this result we know that Russia offers the best salary for their employes  